This is based on this code: https://github.com/codekansas/keras-language-modeling/blob/master/keras_models.py

In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
#Check mongo is running
import pymongo
client = pymongo.MongoClient()
db = client.metrics

## Note - To Get this working:

* Install CUDA and associated libraries, setup path
* Install bleeding edge theano (from src)
* Make sure the THEANO_FLAGS are set correctly via the environment var, or via the ~/.theanorc file
* Install and compile bleeding edge Keras (from src)
* `export KERAS_BACKEND=theano`
* `export KERAS_IMAGE_DIM_ORDERING='th'`
* `sh <project_root>/shell_scipts/setup_environment.sh` to install additional dependencies
* **DO NOT SET UNROLL=True** when creating RNN's - causes max recursion issue

## Trouble-Shooting

* You may need to clean the theano cache. To do so thoroughly, run this command from the shell:
 * `theano-cache purge`

In [3]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

import keras
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, TimeDistributedDense
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging

import datetime

Using Theano backend.


## Load and Pre-Process Essays

In [4]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM-4-SEARN/"
predictions_folder = root_folder + "Predictions/Bi-LSTM-4-SEARN/"
cv_folder = root_folder + "CV_Data_Pickled_SEARN/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [5]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [6]:
test_config = get_config(test_folder)
tagged_essays_test = load_process_essays(**test_config)

226 files found
226 essays processed


In [7]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-07-29 16:09:39.016635


In [8]:
from numpy.random import shuffle
#shuffle(tagged_essays)

## Prepare Tags

In [10]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

for essay in tagged_essays_test:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit() or t == "explicit"))
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))

vtags = set(regular_tags)
vtags.add(EMPTY_TAG)

len(unique_words)

1677

In [11]:
sorted(vtags)

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Empty',
 'explicit']

# Transform Essays into Training Data (Word Ids)

* Computes `xs`, `ys`, `ys_bytag` and `seq_lens`
* `ys_bytag` includes **all tags** and does **not** focus only on the most common tag
* `ys` only includes the most common tag (so we can use cross entropy)
* `seq_lens` is without the start and end tags included (so we have to map back and forth to maintain mappings)
* `ys_bytag` also excludes the START and END tokens

## Get Max Sequence Length, Generate All Ids

In [12]:
ix2tag = {}
for ix, t in enumerate(vtags):
    ix2tag[ix] = t
    
generator = idGen(seed=1) # important as we zero pad sequences

maxlen = 0
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)
        
for essay in tagged_essays_test:
    for sentence in essay.sentences:
        for word, tags in sentence:
            id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
        maxlen = max(maxlen, len(sentence) + 2)

def ids2tags(ids):
    return [generator.get_key(j) for j in ids]

def lbls2tags(ixs):
    return [ix2tag[ix] for ix in ixs]
        
maxlen

141

In [19]:
[(t.ljust(30),t.replace("Causer:","").replace("Result:","")) for t in sorted(tag_freq.keys())]

[('1                             ', '1'),
 ('11                            ', '11'),
 ('12                            ', '12'),
 ('13                            ', '13'),
 ('14                            ', '14'),
 ('2                             ', '2'),
 ('3                             ', '3'),
 ('4                             ', '4'),
 ('5                             ', '5'),
 ('50                            ', '50'),
 ('5b                            ', '5b'),
 ('6                             ', '6'),
 ('7                             ', '7'),
 ('Anaphor                       ', 'Anaphor'),
 ('COMPILED                      ', 'COMPILED'),
 ('Causer                        ', 'Causer'),
 ('Causer:1                      ', '1'),
 ('Causer:1->Result:11           ', '1->11'),
 ('Causer:1->Result:13           ', '1->13'),
 ('Causer:1->Result:14           ', '1->14'),
 ('Causer:1->Result:2            ', '1->2'),
 ('Causer:1->Result:3            ', '1->3'),
 ('Causer:1->Result:4            '

In [21]:
START = "<start>"
END   = "<end>"

def get_training_data(tessays):
    # outputs
    xs = []
    ys = []
    seq_lens = []
    ys_bytag_concept_sent = defaultdict(list)

    # cut texts after this number of words (among top max_features most common words)
    for essay in tessays:
        for sentence in essay.sentences:
            row = []
            y_found = False
            y_seq = []
            unique_tags = set() # get all unique tags in sentence
            for word, tags in [(START, set())] + sentence + [(END, set())]:
                id = generator.get_id(word) #starts at 0, but 0 used to pad sequences
                row.append(id)
                
                # Make sure to include Causer:<num> and Result:<num> tags
                tags = set([t.replace("Causer:","").replace("Result:","") for t in tags])
                unique_tags.update(tags)
                
                # remove unwanted tags, filter to concept tags
                concept_tags = vtags.intersection(tags)

                # encode ys with most common tag only
                if len(concept_tags) > 1:
                    # if multiple, choose concept over explicit
                    if "explicit" in concept_tags:
                        concept_tags.remove("explicit")
                    most_common = max(concept_tags, key=lambda t: tag_freq[t])
                    concept_tags = set([most_common])
                if len(concept_tags) == 0:
                    concept_tags.add(EMPTY_TAG)

                one_hot = []
                for t in vtags:
                    if t in concept_tags:
                        one_hot.append(1)
                    else:
                        one_hot.append(0)
                y_seq.append(one_hot)
                #end for each word

            # sentence level tags
            for tag in vtags:
                if tag in unique_tags:
                    ys_bytag_concept_sent[tag].append(1)
                else:
                    ys_bytag_concept_sent[tag].append(0)            
            
            seq_lens.append(len(row)-2)
            ys.append(y_seq)
            xs.append(row)
    
    xs = sequence.pad_sequences(xs, maxlen=maxlen)
    ys = sequence.pad_sequences(ys, maxlen=maxlen)
    assert xs.shape[0] == ys.shape[0], "Sequences should have the same number of rows"
    assert xs.shape[1] == ys.shape[1] == maxlen, "Sequences should have the same lengths"
    return xs, ys, ys_bytag_concept_sent, seq_lens

## Create Train - Test Split

In [22]:
#Helper Functions
def collapse_results(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = pred_ys[1:-1]
        for pred_tag in pred_ys:
            pred_ys_by_tag[pred_tag].append(1)
            # for all other tags, a 0
            for tag in(vtags - set([EMPTY_TAG, pred_tag])):
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag

In [23]:
#Helper Functions
def collapse_results_sentence_level(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    pred_ys_by_tag = defaultdict(list)
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = set(pred_ys[1:-1])
        for tag in vtags:
            if tag == EMPTY_TAG:
                continue
            if tag in pred_ys:
                pred_ys_by_tag[tag].append(1)
            else:
                pred_ys_by_tag[tag].append(0)
        if EMPTY_TAG in pred_ys_by_tag:
            del pred_ys_by_tag[EMPTY_TAG]
    return pred_ys_by_tag


In [24]:
def train_dev_split(lst, dev_split, randomize=True):
    # random shuffle
    if randomize:
        shuffle(lst)
    num_training = int((1.0 - dev_split) * len(lst))
    return lst[:num_training], lst[num_training:]

In [29]:
e = tagged_essays[0]
e.sentences[0]

[('what', set()),
 ('leads', set()),
 ('to', set()),
 ('differences', set()),
 ('in', set()),
 ('the', set()),
 ('rates', set()),
 ('of', set()),
 ('coral', {'50'}),
 ('bleaching', {'50'}),
 ('.', set())]

In [30]:
%%time
# use this name for a different function later
from CrossValidation import cross_validation as cv

folds = cv(tagged_essays, CV_FOLDS)
fold2training_data = {}
fold2dev_data = {}
fold2test_data = {}

fold2training_essays = {}
fold2dev_essays = {}
fold2test_essays = {}

for i, (essays_TD, essays_VD) in enumerate(folds):
    # further split into train and dev test
    essays_train, essays_dev = train_dev_split(essays_TD, DEV_SPLIT)
    fold2training_data[i] = get_training_data(essays_train)
    fold2dev_data[i]      = get_training_data(essays_dev)
    # Test Data
    fold2test_data[i]     = get_training_data(essays_VD)
    
    # also store essays
    fold2training_essays[i] = essays_train
    fold2dev_essays[i]      = essays_dev
    fold2test_essays[i]     = essays_VD

CPU times: user 6.25 s, sys: 383 ms, total: 6.64 s
Wall time: 6.63 s


In [31]:
# Make sure the generator is incremented on the test data too
_,_,_,_ = get_training_data(tagged_essays)
_,_,_,_ = get_training_data(tagged_essays_test)

In [32]:
with open(cv_folder + "td.dill", "wb") as f:
    dill.dump(fold2training_data, f)

with open(cv_folder + "td_essays.dill", "wb") as f:
    dill.dump(fold2training_essays, f)

with open(cv_folder + "devd.dill", "wb") as f:
    dill.dump(fold2dev_data, f)

with open(cv_folder + "devd_essays.dill", "wb") as f:
    dill.dump(fold2dev_essays, f)
    
with open(cv_folder + "vd.dill", "wb") as f:
    dill.dump(fold2test_data, f)
    
with open(cv_folder + "vd_essays.dill", "wb") as f:
    dill.dump(fold2test_essays, f)
    
with open(cv_folder + "generator.dill", "wb") as f:
    dill.dump(generator, f)

with open(cv_folder + "vtags.dill", "wb") as f:
    dill.dump(vtags, f)

In [33]:
generator.get_id("coral"), generator.get_key(generator.get_id("coral"))

(9, 'coral')

## Load Glove 100 Dim Embeddings

In [34]:
# see /Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/WordVectors/pickle_glove_embedding.py
# for creating pre-filtered embeddings file
import pickle, os
from numpy.linalg import norm

embeddings_file = "/Users/simon.hughes/data/word_embeddings/glove.6B/cb_dict_glove.6B.100d.txt"
# read data file
with open(embeddings_file, "rb+") as f:
    cb_emb_index = pickle.load(f)

In [35]:
missed = set()
for wd in unique_words:
    if wd not in cb_emb_index:
        missed.add(wd)
print(len(missed), len(unique_words), 100.0 * round(len(missed)/  len(unique_words),4), "%")

42 1677 2.5 %


### Construct Embedding Matrix

In [36]:
EMBEDDING_DIM = list(cb_emb_index.values())[0].shape[0]

def get_embedding_matrix(words, idgenerator, max_features, init='uniform', unit_length=False):
    embedding_dim = list(cb_emb_index.values())[0].shape[0]
    # initialize with a uniform distribution
    if init == 'uniform':
        # NOTE: the max norms for these is quite low relative to the embeddings
        embedding_matrix = np.random.uniform(low=-0.05, high=0.05,size=(max_features, embedding_dim))
    elif init =='zeros':
        embedding_matrix = np.zeros(shape=(max_features, embedding_dim), dtype=np.float32)
    elif init == 'normal':
        embedding_matrix = np.random.normal(mean, sd, size=(max_features, embedding_dim))
    else:
        raise Exception("Unknown init type")
    for word in words:
        i = idgenerator.get_id(word)
        embedding_vector = cb_emb_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    if unit_length:
        norms = np.linalg.norm(embedding_matrix, axis=1,keepdims=True)
        # remove 0 norms to prevent divide by zero
        norms[norms == 0.0] = 1.0
        embedding_matrix = embedding_matrix / norms
    return embedding_matrix

In [37]:
def score_predictions(model, xs, ys_by_tag, seq_len):
    preds = model.predict_classes(xs, batch_size=batch_size, verbose=0)   
    pred_ys_by_tag = collapse_results_sentence_level(seq_len, preds)
    class2metrics = ResultsProcessor.compute_metrics(ys_by_tag, pred_ys_by_tag)
    micro_metrics = micro_rpfa(class2metrics.values())
    return micro_metrics, pred_ys_by_tag

In [38]:
def pivot_predictions_to_dict(preds):
    pred_ys_by_tag = defaultdict(list)
    for i in range(preds.shape[0]):
        row = preds[i]
        for tag_ix, pred in enumerate(row):
            tag = ix2crtag[tag_ix]
            pred_ys_by_tag[tag].append(pred)
    return pred_ys_by_tag

In [39]:
def score_predictions_sent_level(model, xs, ys_by_tag, seq_len):
    preds = model.predict(xs, batch_size=batch_size, verbose=0)
    preds = np.where(preds >= 0.5, 1, 0)
    pred_ys_by_tag = pivot_predictions_to_dict(preds)
    class2metrics = ResultsProcessor.compute_metrics(ys_by_tag, pred_ys_by_tag)
    micro_metrics = micro_rpfa(class2metrics.values())
    return micro_metrics, pred_ys_by_tag

In [40]:
from keras.layers import Bidirectional
from datetime import datetime

def get_ts():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

def get_file_ts():
    return datetime.now().strftime('%Y%m%d_%H%M%S_%f')

embedding_size = EMBEDDING_DIM
hidden_size    = 128
out_size = len(vtags)
batch_size = 128

get_ts(), get_file_ts()

('2017-07-29 16:21:46.149002', '20170729_162146_149030')

## Train Bi-Directional LSTM With Glove Embeddings

In [41]:
max_features=len(generator.get_ids())+2 #Need plus one maybe due to masking of sequences

In [42]:
def get_file_signature(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    lcls = locals()
    s = ""
    for k, val in sorted(lcls.items(), key = lambda tpl: (0,tpl[0]) if tpl[0] == 'fold_ix' else (1,tpl[0])):
        if val is not None:
            s += "{key}-{val}_".format(key=k, val=str(val))
    return s[:-1]

def get_file_name(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    fsig = get_file_signature(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
    return models_folder + fsig + ".h5"

get_file_name(0, True, True, 2, "sum", hidden_size)

'/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Models/Bi-LSTM-4-SEARN/fold_ix-0_bi_directional-True_hidden_size-128_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.h5'

In [43]:
# merge_mode is Bi-Directional only
def evaluate_fold(fold_ix, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):

    if use_pretrained_embedding:
        embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
        embedding_layer = Embedding(max_features,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=True,
                                mask_zero=True) # If false, initialize unfound words with all 0's
    else:
        embedding_layer = Embedding(max_features, embedding_size, input_length=maxlen, trainable=True, mask_zero=True)

    if bi_directional:
        rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
    else:
        rnn_layer_fact = lambda : GRU(hidden_size, return_sequences=True, consume_less="cpu")
        
    model = Sequential()
    model.add(embedding_layer)
    for i in range(num_rnns):
        model.add(rnn_layer_fact())

    model.add(TimeDistributedDense(out_size))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")
    
    X_train, y_train, train_ys_bytag_con_sent, seq_len_train = fold2training_data[fold_ix]
    X_dev,   y_dev,   dev_ys_bytag_con_sent,   seq_len_dev   = fold2dev_data[fold_ix]
    X_test,  y_test,  test_ys_bytag_con_sent,  seq_len_test  = fold2test_data[fold_ix]

    # init loop vars
    f1_scores = [-1]
    num_since_best_score = 0
    patience = 3
    best_weights = None

    for i in range(30):
    #for i in range(10):
        print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
        epochs = 1 # epochs per training instance
        results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, validation_split=0.0, verbose=0)
        micro_metrics,_ = score_predictions(model, X_dev, dev_ys_bytag_con_sent, seq_len_dev)

        #print(micro_metrics)
        #print()

        f1_score = micro_metrics.f1_score
        best_f1_score = max(f1_scores)
        if f1_score <= best_f1_score:
            num_since_best_score += 1
        else: # score improved
            num_since_best_score = 0
            best_weights = model.get_weights()

        f1_scores.append(f1_score)
        if num_since_best_score >= patience:
            #print("Too long since an improvement, stopping")
            break
    
    print("Fold[{ix}] - Best F1 Score={f1}".format(ix=fold_ix, f1=best_f1_score))
    
    # load best weights
    model.set_weights(best_weights)
    return model

## Hyper Param Tuning

In [44]:
def cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    fold2model = {}
    for i in range(CV_FOLDS):
        model = evaluate_fold(i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
        fname = get_file_name(i, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
        model.save(fname)    
        fold2model[i] = model
    return fold2model

In [46]:
%%time
import warnings
warnings.filterwarnings("ignore")

i = 0

use_pretrained_embedding = True
bi_directional = True
num_rnns = 2
merge_mode = "sum"
hidden_size = 256

"""
use_pretrained_embedding = True
bi_directional = False
num_rnns = 1
merge_mode = "sum"
hidden_size = 32
"""

i += 1
print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
      .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))
fold2model = cross_validation(use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
#print("MicroF1={micro_f1}".format(micro_f1=micro_f1))
print("[{i}] Params {ts} - Embeddings={use_pretrained_embedding}, Bi-Direct={bi_directional} Num_Rnns={num_rnns} Hidden_Size={hidden_size}"\
      .format(i=i, ts=get_ts(), use_pretrained_embedding=use_pretrained_embedding, bi_directional=bi_directional, num_rnns=num_rnns, hidden_size=hidden_size))

[1] Params 2017-07-29 20:28:45.664809 - Embeddings=True, Bi-Direct=True Num_Rnns=2 Hidden_Size=256
2017-07-29 20:28:45.929278: Epoch=0
2017-07-29 20:34:13.280074: Epoch=1
2017-07-29 20:39:26.790112: Epoch=2
2017-07-29 20:44:35.131474: Epoch=3
2017-07-29 20:49:43.910672: Epoch=4
2017-07-29 20:55:12.875216: Epoch=5
2017-07-29 20:59:42.766856: Epoch=6
2017-07-29 21:03:57.162425: Epoch=7
2017-07-29 21:08:11.036494: Epoch=8
2017-07-29 21:12:28.772221: Epoch=9
2017-07-29 21:17:26.391252: Epoch=10
2017-07-29 21:22:56.179849: Epoch=11
2017-07-29 21:28:02.685775: Epoch=12
2017-07-29 21:33:05.558024: Epoch=13
2017-07-29 21:38:12.737116: Epoch=14
2017-07-29 21:42:37.997327: Epoch=15
2017-07-29 21:46:51.163560: Epoch=16
Fold[0] - Best F1 Score=0.8559542381695266
2017-07-29 21:51:06.086218: Epoch=0
2017-07-29 21:55:48.495094: Epoch=1
2017-07-29 22:00:05.547044: Epoch=2
2017-07-29 22:04:49.303562: Epoch=3
2017-07-29 22:09:59.568002: Epoch=4
2017-07-29 22:15:24.206718: Epoch=5
2017-07-29 22:21:07.075

In [47]:
predicts_by_fold = {}
for fold_ix in range(CV_FOLDS):
    X_test,  y_test,  test_ys_bytag_con_sent,  seq_len_test  = fold2test_data[fold_ix]
    model = fold2model[fold_ix]
    probs = model.predict_classes(X_test)
    predicts_by_fold[fold_ix] = probs

1492/1492 [==============================] - 14s     

In [48]:
def get_predictions_fname(fold, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size):
    fsig = get_file_signature(fold, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
    return predictions_folder + fsig + ".dill"

for fold, preds in predicts_by_fold.items():
    fname = get_predictions_fname(fold, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size)
    with open(fname, "wb") as f:
        dill.dump(preds, f)

In [49]:
#Helper Functions
def predictions_to_tags(seq_lens, preds):
    assert len(seq_lens) == preds.shape[0], "Axis 1 size does not align"
    sentence_tags = []
    for i in range(len(seq_lens)):
        row_ixs = preds[i,:]
        len_of_sequence = seq_lens[i] + 2
        # sequences are padded from the left, take the preds from the end of the seq
        pred_ys = [ix2tag[j] for j in row_ixs[-len_of_sequence:]]
        # skip the start and end label
        pred_ys = pred_ys[1:-1]
        sentence_tags.append(pred_ys)
    return sentence_tags

## Assign Predicted Tags to Essay Objects

In [50]:
def assign_tags_to_essays(essays, preds, seq_len):
    pred_tags = predictions_to_tags(seq_len, preds)
    sent_ix = 0
    for essay in essays:
        ptagged_sentences = []
        for sent in essay.sentences:
            ptags = pred_tags[sent_ix]
            assert len(ptags) == len(sent), "Sentence and tags don't align - ntags %i , len(sentence) %i" % ((len(ptags),len(sent)))
            ptagged_sentences.append(ptags)
            sent_ix += 1
        assert len(ptagged_sentences) == len(essay.sentences), "Lens differ"
        essay.pred_tagged_sentences = ptagged_sentences
    assert sent_ix == len(pred_tags), "Predictions don't align with sequence lens"

all_ptagged_essays = []
for fold in fold2test_essays.keys():
    essays = fold2test_essays[fold]
    preds = predicts_by_fold[fold]
    _,_,_,seq_len = fold2test_data[fold]
    assign_tags_to_essays(essays, preds, seq_len)
    all_ptagged_essays.extend(essays)

fname = predictions_folder + "essays_train_" + get_file_signature(None, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size) + ".dill"
with open(fname, "wb") as f:
    dill.dump(all_ptagged_essays, f)

In [51]:
assert len(all_ptagged_essays) == len(tagged_essays)

## Get Test Data Predictions

In [ ]:
# merge_mode is Bi-Directional only
def evaluate_test(num_rnns, merge_mode, hidden_size):
    embedding_matrix = get_embedding_matrix(unique_words, generator, max_features, init='uniform', unit_length=False)
    embedding_layer = Embedding(max_features,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=True,
                            mask_zero=True) # If false, initialize unfound words with all 0's
    rnn_layer_fact = lambda : Bidirectional(GRU(hidden_size, return_sequences=True, consume_less="cpu"), merge_mode=merge_mode)
        
    model = Sequential()
    model.add(embedding_layer)
    for i in range(num_rnns):
        model.add(rnn_layer_fact())

    model.add(TimeDistributedDense(out_size))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', sample_weight_mode="temporal")
    
    essays_train, essays_dev = train_dev_split(tagged_essays, DEV_SPLIT)
    X_train, y_train,  train_ys_bytag_con_sent,  seq_len_train = get_training_data(essays_train)
    X_dev,   y_dev,    dev_ys_bytag_con_sent,    seq_len_dev   = get_training_data(essays_dev)

    # init loop vars
    f1_scores = [-1]
    num_since_best_score = 0
    patience = 3
    best_weights = None

    for i in range(30):
    #for i in range(10):
        print("{ts}: Epoch={epoch}".format(ts=get_ts(), epoch=i))
        results = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, validation_split=0.0, verbose=0)
        micro_metrics,_ = score_predictions(model, X_dev, dev_ys_bytag_con_sent, seq_len_dev)

        print(micro_metrics)
        print()

        f1_score = micro_metrics.f1_score
        best_f1_score = max(f1_scores)
        if f1_score <= best_f1_score:
            num_since_best_score += 1
        else: # score improved
            num_since_best_score = 0
            best_weights = model.get_weights()

        f1_scores.append(f1_score)
        if num_since_best_score >= patience:
            #print("Too long since an improvement, stopping")
            break
    
    print("Fold[{ix}] - Best F1 Score={f1}".format(ix=fold_ix, f1=best_f1_score))
    
    # load best weights
    model.set_weights(best_weights)
    return model

In [ ]:
#test_model = evaluate_test(2, "sum", 256)
print(num_rnns, merge_mode, hidden_size)
test_model = evaluate_test(num_rnns, merge_mode, hidden_size)

2 sum 256
2017-07-30 11:08:03.400337: Epoch=0


In [ ]:
X_test,  y_test,   test_ys_bytag_con_sent,   seq_len_test = get_training_data(tagged_essays_test)
test_preds = test_model.predict_classes(X_test)

In [ ]:
test_preds.shape, len(test_ys_bytag_con_sent['4'])

In [ ]:
assign_tags_to_essays(tagged_essays_test, test_preds, seq_len_test)

fname = predictions_folder + "essays_test_" + get_file_signature(None, use_pretrained_embedding, bi_directional, num_rnns, merge_mode, hidden_size) + ".dill"
with open(fname, "wb") as f:
    dill.dump(tagged_essays_test, f)

In [ ]:
e = tagged_essays_test[0]
list(zip(e.sentences[0],e.pred_tagged_sentences[0]))

In [244]:
len(tagged_essays_test)

226